# Modeling
Experiment different algorithms and pick best model

In [11]:
# Imports
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score
import joblib
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score

In [2]:
# load in split data
X_train = np.load("/Users/rohith/Desktop/fraud-detection-ml/Data/prep/creditcard_X_train.npy")
y_train = np.load("/Users/rohith/Desktop/fraud-detection-ml/Data/prep/creditcard_y_train.npy")
X_val   = np.load("/Users/rohith/Desktop/fraud-detection-ml/Data/prep/creditcard_X_val.npy")
y_val   = np.load("/Users/rohith/Desktop/fraud-detection-ml/Data/prep/creditcard_y_val.npy")
x_test = np.load("/Users/rohith/Desktop/fraud-detection-ml/Data/prep/creditcard_X_test.npy")
y_test   = np.load("/Users/rohith/Desktop/fraud-detection-ml/Data/prep/creditcard_y_test.npy")

print(f"Train shape: {X_train.shape},  Val shape: {X_val.shape}")

Train shape: (192964, 11),  Val shape: (41349, 11)


### Baseline runs

In [3]:
# Dummy
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)
print("Dummy AUC:", roc_auc_score(y_val, dummy.predict_proba(X_val)[:,1]))

Dummy AUC: 0.5


In [4]:
# Shallow Tree
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_train, y_train)
print("Shallow Tree AUC:",
      roc_auc_score(y_val, tree.predict_proba(X_val)[:,1]))


Shallow Tree AUC: 0.6963779748210523


In [5]:
# RandomForrest Classifier
rForrest = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1)

rForrest.fit(X_train, y_train)

prob = rForrest.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, prob)
print("RandomForest (100 trees) ROC AUC:", round(auc, 4))


RandomForest (100 trees) ROC AUC: 0.9447


In [6]:
# LightGBM
lgbm = LGBMClassifier(
    is_unbalance=True,    
    n_estimators=100,     
    learning_rate=0.1, 
    random_state=42,
    n_jobs=-1
)

lgbm.fit(X_train, y_train)

proba = lgbm.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, proba)
print("LightGBM (is_unbalance=True) ROC AUC:", round(auc, 4))

[LightGBM] [Info] Number of positive: 365, number of negative: 192599
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000784 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2805
[LightGBM] [Info] Number of data points in the train set: 192964, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001892 -> initscore=-6.268468
[LightGBM] [Info] Start training from score -6.268468
LightGBM (is_unbalance=True) ROC AUC: 0.907


/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


1. Dummy AUC: 0.500  
2. Shallow Tree AUC: 0.696  
3. RandomForest ROC AUC: 0.945  
4. LightGBM ROC AUC: 0.907

### Tuning Random Forrest Classifier

#### Randomized Search CV

In [7]:
# Parameters to tune
params = {
    "n_estimators": [50, 100, 250, 500, 1000],
    "max_depth": [None, 5, 10, 20],
    "max_features":    ["sqrt", "log2", 0.3, 0.5],
    "min_samples_split":[2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

In [8]:
# cross-validation strategy
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [9]:
# Search configs
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rs = RandomizedSearchCV(
    rf,
    param_distributions=params,
    n_iter=30,               
    scoring="roc_auc",
    cv=cv,
    verbose=1,
    random_state=42
)

In [10]:
# Finding best params
"""
rs.fit(X_train, y_train)
print("Best ROC-AUC (val):", rs.best_score_)
print("Best params:", rs.best_params_)
"""

'\nrs.fit(X_train, y_train)\nprint("Best ROC-AUC (val):", rs.best_score_)\nprint("Best params:", rs.best_params_)\n'

##### Randomized Search Results
Fitting 3 folds for each of 30 candidates, totalling 90 fits  
Best ROC-AUC (val): 0.9720197513738311  
Best params: {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 0.3, 'max_depth': 10}

#### Optuna

In [15]:
def rf_objective(trial):
    params = {
        "n_estimators":    trial.suggest_int("n_estimators", 100, 800),
        "max_depth":       trial.suggest_int("max_depth", 5, 30),
        "max_features":    trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3, 0.5]),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf":  trial.suggest_int("min_samples_leaf", 1, 4),
    }
    clf = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
    # 3-fold CV AUC
    score = cross_val_score(clf, X_train, y_train, cv=3, scoring="roc_auc").mean()
    return score

In [ ]:
# Run optuna study
'''
study = optuna.create_study(direction="maximize")
study.optimize(rf_objective, n_trials=50, show_progress_bar=True)
'''

[I 2025-07-03 17:00:05,164] A new study created in memory with name: no-name-da2d9200-5b90-40aa-ab7f-f01d14a69fb0
Best trial: 0. Best value: 0.958488:   2%|▏         | 1/50 [01:21<1:06:22, 81.27s/it]

[I 2025-07-03 17:01:26,433] Trial 0 finished with value: 0.9584877954973038 and parameters: {'n_estimators': 350, 'max_depth': 29, 'max_features': 'sqrt', 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9584877954973038.


Best trial: 1. Best value: 0.967414:   4%|▍         | 2/50 [08:03<3:35:49, 269.79s/it]

[I 2025-07-03 17:08:08,183] Trial 1 finished with value: 0.9674141407274811 and parameters: {'n_estimators': 760, 'max_depth': 13, 'max_features': 0.5, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.9674141407274811.


Best trial: 2. Best value: 0.969235:   6%|▌         | 3/50 [12:13<3:24:24, 260.94s/it]

...

[I 2025-07-03 18:53:05,897] Trial 47 finished with value: 0.9693337278808242 and parameters: {'n_estimators': 268, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 3}. Best is trial 36 with value: 0.9728786086958349.


Best trial: 36. Best value: 0.972879:  98%|█████████▊| 49/50 [1:54:21<01:07, 67.13s/it]

[I 2025-07-03 18:54:26,956] Trial 48 finished with value: 0.9555777866905908 and parameters: {'n_estimators': 209, 'max_depth': 26, 'max_features': 'sqrt', 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 36 with value: 0.9728786086958349.


Best trial: 36. Best value: 0.972879: 100%|██████████| 50/50 [1:55:17<00:00, 138.35s/it]

[I 2025-07-03 18:55:22,548] Trial 49 finished with value: 0.9710923367988237 and parameters: {'n_estimators': 280, 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 36 with value: 0.9728786086958349.


##### Optuna Results
[I 2025-07-03 18:55:22,548] Trial 49 finished with value: 0.9710923367988237 and parameters: {'n_estimators': 280, 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_split': 3, 'min_samples_leaf': 4}.   
Best is trial 36 with value: 0.9728786086958349.

### Tuning LightGBM

#### Randomized Search CV

In [39]:
# Reload data
X_train = np.load("/Users/rohith/Desktop/fraud-detection-ml/Data/prep/creditcard_X_train.npy")
y_train = np.load("/Users/rohith/Desktop/fraud-detection-ml/Data/prep/creditcard_y_train.npy")

In [40]:
# Params
lgbm_param_dist = {
    "n_estimators":     [100, 300, 500, 800],
    "learning_rate":    [0.01, 0.05, 0.1],
    "max_depth":        [5, 10, 15, None],
    "num_leaves":       [31, 50, 100, 200],
    "scale_pos_weight": [1, 50, 100, 200]
}

In [41]:
# Set up CV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [42]:
# Search configs
lgbm = LGBMClassifier(is_unbalance=True, random_state=42, n_jobs=-1, verbose=-1)
rs = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=lgbm_param_dist,
    n_iter=30,
    scoring="roc_auc",
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

In [ ]:
"""
# Find best params
rs.fit(X_train, y_train)

print("Best LGBM ROC-AUC (val):", rs.best_score_)
print("Best params:", rs.best_params_)
"""

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/valida

Best LGBM ROC-AUC (val): 0.9600263136892746
Best params: {'scale_pos_weight': 1, 'num_leaves': 50, 'n_estimators': 300, 'max_depth': None, 'learning_rate': 0.01}


##### Randomized SearchCV Results
Best LGBM ROC-AUC (val): 0.9600263136892746  
Best params: {'scale_pos_weight': 1, 'num_leaves': 50, 'n_estimators': 300, 'max_depth': None, 'learning_rate': 0.01}

#### Optuna

In [50]:
# Optuna objective
def lgbm_objective(trial):
    params = {
        "n_estimators":     trial.suggest_int("n_estimators", 100, 800),
        "learning_rate":    trial.suggest_loguniform("learning_rate", 1e-3, 0.1),
        "max_depth":        trial.suggest_int("max_depth", 5, 20),
        "num_leaves":       trial.suggest_int("num_leaves", 31, 200),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 100, 1000, log=True)
    }
    clf = LGBMClassifier(**params, random_state=42, n_jobs=-1, verbose=-1)
    # 3-fold CV AUC
    auc = cross_val_score(clf, X_train, y_train, cv=3, scoring="roc_auc").mean()
    return auc

In [ ]:
"""
# Run study
study = optuna.create_study(direction="maximize")
study.optimize(lgbm_objective, n_trials=50, show_progress_bar=True)

# Best trial
print("Best LGBM ROC-AUC (val):", study.best_value)
print("Best params:", study.best_params)
"""

[I 2025-07-03 21:38:04,842] A new study created in memory with name: no-name-3543431d-e1ac-4f22-b359-4b01aa3840e2
  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/82/87xd0yg57r1gc_vh3xv9zrlw0000gn/T/ipykernel_11944/469091344.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate":    trial.suggest_loguniform("learning_rate", 1e-3, 0.1),
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detect

[I 2025-07-03 21:38:42,497] Trial 0 finished with value: 0.9327846994635886 and parameters: {'n_estimators': 778, 'learning_rate': 0.020743938796379792, 'max_depth': 11, 'num_leaves': 180, 'scale_pos_weight': 453.827234469319}. Best is trial 0 with value: 0.9327846994635886.


/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
Best trial: 1. Best value: 0.950028:   4%|▍         | 2/50 [00:52<19:21, 24.20s/it]/var/folders/82/87xd0yg57r1gc_vh3xv9zrlw0000gn/T/ipykernel_11944/469091344.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.

[I 2025-07-03 21:38:57,289] Trial 1 finished with value: 0.9500284613683273 and parameters: {'n_estimators': 260, 'learning_rate': 0.0016774851264175013, 'max_depth': 14, 'num_leaves': 153, 'scale_pos_weight': 483.08341339705095}. Best is trial 1 with value: 0.9500284613683273.


/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
Best trial: 2. Best value: 0.953045:   6%|▌         | 3/50 [00:58<12:31, 15.99s/it]/var/folders/82/87xd0yg57r1gc_vh3xv9zrlw0000gn/T/ipykernel_11944/469091344.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.

...

[I 2025-07-03 21:54:24,111] Trial 47 finished with value: 0.9514397795399 and parameters: {'n_estimators': 295, 'learning_rate': 0.011091938168599965, 'max_depth': 13, 'num_leaves': 110, 'scale_pos_weight': 856.9684351050222}. Best is trial 35 with value: 0.9651582885633744.


/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
Best trial: 35. Best value: 0.965158:  98%|█████████▊| 49/50 [16:45<00:21, 21.74s/it]/var/folders/82/87xd0yg57r1gc_vh3xv9zrlw0000gn/T/ipykernel_11944/469091344.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v

[I 2025-07-03 21:54:49,931] Trial 48 finished with value: 0.9305089988499918 and parameters: {'n_estimators': 430, 'learning_rate': 0.017065639285669856, 'max_depth': 12, 'num_leaves': 136, 'scale_pos_weight': 564.8791499991389}. Best is trial 35 with value: 0.9651582885633744.


/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/rohith/Desktop/fraud-detection-ml/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
Best trial: 35. Best value: 0.965158: 100%|██████████| 50/50 [16:54<00:00, 20.29s/it]

[I 2025-07-03 21:54:59,534] Trial 49 finished with value: 0.9534538918297936 and parameters: {'n_estimators': 162, 'learning_rate': 0.003565961745342314, 'max_depth': 16, 'num_leaves': 127, 'scale_pos_weight': 905.8246548748929}. Best is trial 35 with value: 0.9651582885633744.
Best LGBM ROC-AUC (val): 0.9651582885633744
Best params: {'n_estimators': 487, 'learning_rate': 0.009035849377469507, 'max_depth': 15, 'num_leaves': 138, 'scale_pos_weight': 990.3099112706714}


##### Optuna Results
Best LGBM ROC-AUC (val): 0.9651582885633744  
Best params: {'n_estimators': 487, 'learning_rate': 0.009035849377469507, 'max_depth': 15, 'num_leaves': 138, 'scale_pos_weight': 990.3099112706714}

## Model Performance Summary
| Model                                 | ROC-AUC  |
|---------------------------------------|--------:|
| **Dummy Classifier**                  | 0.500   |
| **Shallow Decision Tree**             | 0.696   |
| **Random Forest (baseline)**          | 0.945   |
| **LightGBM (baseline)**               | 0.907   |

---

### Random Forest Tuning

#### 1. RandomizedSearchCV  
- **Folds × Candidates:** 3 CV × 30 = 90 fits  
- **Best ROC-AUC (val):** 0.9720  
- **Best params:**  
  ```json
  {
    "n_estimators": 500,
    "min_samples_split": 10,
    "min_samples_leaf": 2,
    "max_features": 0.3,
    "max_depth": 10
  }
  ```

#### 2. Optuna
- **Trials:** 50
- **Example trial 49:**
  - value = 0.97109
  - params = { n_estimators: 280, max_depth: 6, max_features: "sqrt", min_samples_split: 3, min_samples_leaf: 4 }
- **Best trial (36):**
  - value = 0.97288
  - params = { n_estimators: 500, max_depth: 10, max_features: 0.3, min_samples_split: 10, min_samples_leaf: 2 }

### LightGBM Tuning

#### 1. RandomizedSearchCV
- **Best ROC-AUC (val):** 0.9600
- **Best params:**
  ```json
  {
    "scale_pos_weight": 1,
    "num_leaves": 50,
    "n_estimators": 300,
    "max_depth": null,
    "learning_rate": 0.01
  }
  ```

#### 2. Optuna
- **Best ROC-AUC (val):** 0.9652
- **Best params:**
  ```json
  {
    "n_estimators": 487,
    "learning_rate": 0.009035849377469507,
    "max_depth": 15,
    "num_leaves": 138,
    "scale_pos_weight": 990.3099112706714
  }
  ```